# 상가(상권)정보

https://www.data.go.kr/dataset/15012005/openapi.do

In [1]:
import os
# API 키는 환경변수에 따로 저장해 놓고 사용
api_key = os.environ['OPEN_API_KEY_STORE']

In [2]:
from urllib.parse import urlencode
from urllib.parse import unquote
from urllib import request
import xml.etree.ElementTree as ET

class API:
  def __init__(self, key):
    self._key = unquote(key)

  def http_get(self, endpoint, params):
    params_with_key = params.copy()
    params_with_key['ServiceKey'] = unquote(self._key)
    url = endpoint + '?' + urlencode(params_with_key)
    with request.urlopen(url) as f:
      res = f.read().decode('utf-8')
      return ET.fromstring(res)

api = API(api_key)

In [3]:
xml = api.http_get(
    'http://apis.data.go.kr/B553077/api/open/sdsc/baroApi',
    {'resId': 'storezone'}
)

In [4]:
# https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
from collections import defaultdict

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [5]:
response_dict = etree_to_dict(xml)
response_dict = response_dict['response']
header = response_dict['header']
header

{'columns': '상권번호,상권명,시도코드,시도명,시군구코드,시군구명,상권면적,좌표개수,좌표값,데이터기준일자',
 'description': '소상공인시장진흥공단 주요상권',
 'resultCode': '00',
 'resultMsg': 'NORMAL SERVICE'}

In [6]:
df_cols = header['columns']

In [7]:
import pandas as pd

result = pd.DataFrame.from_dict(response_dict['body']['items']['item'])
result.shape

(1170, 10)

In [8]:
result.head()

,coordNum,coords,ctprvnCd,ctprvnNm,mainTrarNm,signguCd,signguNm,stdrDt,trarArea,trarNo
0,24,"POLYGON ((127.015141 37.486601, 127.014701 37....",11,서울특별시,서울 서초구 남부터미널역_2,11650,서초구,2015-12-17,85529,1845
1,39,"POLYGON ((127.015723 37.487831, 127.016185 37....",11,서울특별시,서울 서초구 남부터미널역_3,11650,서초구,2015-12-17,48833.6,1849
2,24,"POLYGON ((126.983867 37.487274, 126.983876 37....",11,서울특별시,서울 동작구 총신대입구역,11650,서초구,2015-12-17,43176,1850
3,7,"POLYGON ((127.035961 37.46077, 127.037071 37.4...",11,서울특별시,양재 화물트럭터미널앞_1,11650,서초구,2015-12-17,31097.1,1744
4,66,"POLYGON ((127.031755 37.488972, 127.031543 37....",11,서울특별시,양재역_3,11650,서초구,2015-12-17,119300.7,1851
